In [9]:
import numpy as np
import pandas as pd
from collections import defaultdict

import pymongo
from pymongo import MongoClient

import time
import requests
import os
import re
from bs4 import BeautifulSoup
import json

In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
global chrome_options
PROXY = ''
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--proxy-server=%s' % PROXY)

In [39]:
import time
from selenium import webdriver

driver = webdriver.Chrome('/Users/CEO/Downloads/chromedriver_2')  # Optional argument, if not specified will search path.
driver.get('http://www.google.com/xhtml');

In [40]:
time.sleep(5) # Let the user actually see something!

In [42]:
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()

In [44]:
time.sleep(5) # Let the user actually see something!
driver.quit()

In [2]:
webpage = requests.get('https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/1849/Stages/3115/Fixtures/England-Premier-League-2009-2010')

In [3]:
soup = BeautifulSoup(webpage.text, 'html.parser')
soup

<html style="height:100%"><head><meta content="NOINDEX, NOFOLLOW" name="ROBOTS"/><meta content="telephone=no" name="format-detection"/><meta content="initial-scale=1.0" name="viewport"/><meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/><script src="/_Incapsula_Resource?SWJIYLWA=719d34d31c8e3a6e6fffd425f7e032f3" type="text/javascript"></script></head><body style="margin:0px;height:100%"><iframe frameborder="0" height="100%" marginheight="0px" marginwidth="0px" src="/_Incapsula_Resource?CWUDNSAI=20&amp;xinfo=6-24589526-0%200NNN%20RT%281542706798603%20187%29%20q%280%20-1%20-1%200%29%20r%280%20-1%29%20B15%284%2c200%2c0%29%20U18&amp;incident_id=118000540321429844-221151211858297702&amp;edet=15&amp;cinfo=04000000" width="100%">Request unsuccessful. Incapsula incident ID: 118000540321429844-221151211858297702</iframe></body></html>

In [62]:
def get_html(season_url, months):
    driver_s = webdriver.Chrome('/Users/CEO/Downloads/chromedriver_2')
    html_list = []
    
    while True:
        try:
            driver_s.get(season_url)
            break
        except:
            driver_s.close()
            driver_s = webdriver.Chrome('/Users/CEO/Downloads/chromedriver_2')

    for month in months:
        time.sleep(5)
        fixture_html = driver_s.page_source
        fixture_soup = BeautifulSoup(fixture_html, 'html.parser')
        html_list.append(fixture_soup)
        if month != months[-1]:
            while True:
                try:
                    driver_s.find_element_by_css_selector('#date-controller > a.previous.button.ui-state-default.rc-l.is-default').click()
                    break
                except:
                    driver_s.refresh()
                    time.sleep(10)

    driver_s.close()
    return html_list

def get_result(data_dict, html_list):
    head = 'https://www.whoscored.com'
    reports = []
    for soup in html_list:
        tags_home = soup.select('#tournament-fixture > tbody > tr > td.team.home')
        tags_result = soup.select('#tournament-fixture > tbody > tr > td.result')
        tags_away = soup.select('#tournament-fixture > tbody > tr > td.team.away')
        tags_match_report = soup.select('#tournament-fixture > tbody > tr > td.toolbar.right > a.match-link.match-report.rc')
        
        for tag_home,tag_result, tag_away, tag_match_report in zip(tags_home, tags_result, tags_away, tags_match_report):
            data_dict['home_teams'].append(tag_home.contents[-1].text)
            
            result = tag_result.text
            array = result.split(':')
            data_dict['results'].append(int(array[0]) - int(array[1]))
            
            data_dict['away_teams'].append(tag_away.contents[0].text)
            data_dict['id'].append(int(tag_match_report['href'].split('/')[2]))
            
            reports.append((head + tag_match_report['href']).replace('Show', 'MatchReport'))
    return data_dict, reports

In [49]:
url_09_10 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/1849/Stages/3115/Fixtures/England-Premier-League-2009-2010'
url_10_11 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/2458/Stages/4345/Fixtures/England-Premier-League-2010-2011'
url_11_12 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/2935/Stages/5476/Fixtures/England-Premier-League-2011-2012'
url_12_13 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/3389/Stages/6531/Fixtures/England-Premier-League-2012-2013'
url_13_14 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/3853/Stages/7794/Fixtures/England-Premier-League-2013-2014'
url_14_15 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/4311/Stages/9155/Fixtures/England-Premier-League-2014-2015'
url_15_16 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/5826/Stages/12496/Fixtures/England-Premier-League-2015-2016'
url_16_17 = 'https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/6335/Stages/13796/Fixtures/England-Premier-League-2016-2017'
urls = [url_09_10, url_10_11, url_11_12, url_12_13, url_13_14, url_14_15, url_15_16, url_16_17]
months = ['May', 'April', 'March', 'Feb', 'Jan', 'Dec', 'Nov', 'Oct', 'Sep', 'Aug']
df_EPL = pd.DataFrame()

In [50]:
html_list = get_html(url_16_17, months)

In [53]:
len(html_list)

10

In [59]:
html_list[1]

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html lang="en" xmlns="http://www.w3.org/1999/xhtml"><head><iframe allowtransparency="true" frameborder="0" height="0" hspace="0" id="25375a4102aa982" marginheight="0" marginwidth="0" sandbox="allow-scripts allow-same-origin" scrolling="no" src="//acdn.adnxs.com/ib/static/usersync/v3/async_usersync.html" style="height:0px;width:0px;display:none;" vspace="0" width="0">
</iframe><iframe allowtransparency="true" frameborder="0" height="0" hspace="0" id="2428ef8acb912b6" marginheight="0" marginwidth="0" sandbox="allow-scripts allow-same-origin" scrolling="no" src="//acdn.adnxs.com/ib/static/usersync/v3/async_usersync.html" style="height:0px;width:0px;display:none;" vspace="0" width="0">
</iframe><iframe allowtransparency="true" frameborder="0" height="0" hspace="0" id="236dd996235fca6" marginheight="0" marginwidth="0" sandbox="allow-scripts allow-same-origin" scrolling="no" src="https://eus.rubiconpr

In [60]:
data_dict = defaultdict(list)

In [61]:
data_dict

defaultdict(list, {})

In [142]:
'''def get_result(data_dict, html_list):
    head = 'https://www.whoscored.com'
    reports = []
    for soup in html_list:
        tags_home = soup.select('#tournament-fixture > tbody > tr > td.team.home')
        tags_result = soup.select('#tournament-fixture > tbody > tr > td.result')
        tags_away = soup.select('#tournament-fixture > tbody > tr > td.team.away')
        tags_match_report = soup.select('#tournament-fixture > tbody > tr > td.toolbar.right > a.match-link.match-report.rc')
        
        for tag_home,tag_result, tag_away, tag_match_report in zip(tags_home, tags_result, tags_away, tags_match_report):
            data_dict['home_teams'].append(tag_home.contents[-1].text)
            
            result = tag_result.text
            array = result.split(':')
            data_dict['results'].append(int(array[0]) - int(array[1]))
            
            data_dict['away_teams'].append(tag_away.contents[0].text)
            data_dict['id'].append(int(tag_match_report['href'].split('/')[2]))
            
            reports.append((head + tag_match_report['href']).replace('Show', 'MatchReport'))
    return data_dict, reports'''

head = 'https://www.whoscored.com'
reports = []
for soup in html_list:
    tags_home = soup.select('#tournament-fixture > tbody > tr > td.team.home')
    tags_result = soup.select('#tournament-fixture > tbody > tr > td.result')
    tags_away = soup.select('#tournament-fixture > tbody > tr > td.team.away')
    tags_match_report = soup.select('#tournament-fixture > tbody > tr > td.toolbar.right > a.match-link.match-report.rc')

print(soup.find_all('td', class_="team home winner")[0])
print()
print(tags_home[0])
print()
print(tags_home[0].contents)
print()
print(tags_home[0].contents[0])
print()
print(tags_home[0].contents[-1])
print()
print(tags_home[0].contents[0].text)
print()
print(tags_home[0].contents[-1].text)
print()
print()
print(tags_result[0])  
print()
print(soup.find_all('a', class_="result-1 rc")[0])
print()
print(tags_result[0].text)  
print()
print(int(tags_result[0].text.split(':')[0]) - 
      int(tags_result[0].text.split(':')[1]))  
print()
print(soup.find_all('td', class_="team away")[0])
print()
print(tags_away[0])  
print()
print(tags_away[0].contents[0].text)
print()
print(soup.findAll('a', class_="match-link match-report rc")[0])  
print()
print(tags_match_report[0])  
print()
print(int(tags_match_report[0]['href'].split('/')[2]))
print()
r=[]
print((tags_match_report[0]['href']).replace('MatchReport', 'Show'))

<td class="team home winner" data-id="214"><a class="team-link " href="/Teams/214/Show/England-Hull" title="">Hull</a></td>

<td class="team home winner" data-id="214"><a class="team-link " href="/Teams/214/Show/England-Hull" title="">Hull</a></td>

[<a class="team-link " href="/Teams/214/Show/England-Hull" title="">Hull</a>]

<a class="team-link " href="/Teams/214/Show/England-Hull" title="">Hull</a>

<a class="team-link " href="/Teams/214/Show/England-Hull" title="">Hull</a>

Hull

Hull


<td class="result"><a class="result-1 rc" href="/Matches/1080512/Live/England-Premier-League-2016-2017-Hull-Leicester">2 : 1</a></td>

<a class="result-1 rc" href="/Matches/1080512/Live/England-Premier-League-2016-2017-Hull-Leicester">2 : 1</a>

2 : 1

1

<td class="team away" data-id="14"><a class="team-link " href="/Teams/14/Show/England-Leicester" title="">Leicester</a></td>

<td class="team away" data-id="14"><a class="team-link " href="/Teams/14/Show/England-Leicester" title="">Leicester</a></t